# Problem 9.15

**Fundamentals of Solar Cells and Photovoltaic Systems Engineering**

**Solutions Manual - Chapter 9**

**Problem 9.15**

**Consider a PV generator composed of PV modules TrinaTSM240PA05 which will be installed on a rooftop with 24$^{\circ}$ tilt angle and south oriented in Los Angeles, California (34.052$^{\circ}$N, 118.243$^{\circ}$W)**

**The capacity of the PV generator will be chosen to match, on average, the current electricity consumption of the dwelling, which is 3,000 kWh per year.**

**(a) Assuming a performance ratio PR=80%, estimate the number of PV modules required for the PV generator.**

**(b) Selecting the inverter 'ABB__PVI_3_0_OUTD_S_US__208V_' from the SAM database, configure the PV generator by connecting the PV modules in series and parallel to ensure compatibility with the inverter. For the cell temperature estimation, assume that the PV module is glass-glass and is mounted on an open rack.**

We will use the packages [pvlib](https://pvlib-python.readthedocs.io/en/stable/), [pandas](https://pandas.pydata.org/) and [matplotlib.pyplot](https://matplotlib.org/stable/index.html) to plot the results. We will also use the package [pytz](https://pypi.org/project/pytz/) to determine the time zone of Los Angeles.

In [1]:
import pvlib
import pandas as pd
import matplotlib.pyplot as plt
import pytz
import math

We start by defining the location, date, and time. 

In [2]:
# Los Angeles, California
lat, lon =  34.052, -118.243

tz = pytz.country_timezones('US')[20] 

# location
location = pvlib.location.Location(lat, lon, tz=tz)

We retrieve typical meteorological year (TMY) data from PVGIS.

In [3]:
tmy, _, _, _ = pvlib.iotools.get_pvgis_tmy(latitude=lat, longitude=lon, map_variables=True)

tmy.index = tmy.index.tz_convert(tz) #use local time

We retrieve the PV modules specifications from the database at the [NREL SAM (System Advisory Monitoring)](https://sam.nrel.gov/). 

In [4]:
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')

module = sandia_modules['Trina_TSM_240PA05__2013_']

module_pmp=module['Impo']*module['Vmpo']

print("Module power = " + str(round(module_pmp,1)) + " W")

Module power = 241.1 W


In [5]:
module

Vintage                                                          2013
Area                                                             1.63
Material                                                        mc-Si
Cells_in_Series                                                    60
Parallel_Strings                                                    1
Isco                                                           8.8449
Voco                                                          36.8926
Impo                                                           8.2955
Vmpo                                                           29.066
Aisc                                                           0.0004
Aimp                                                          -0.0003
C0                                                             1.0116
C1                                                            -0.0116
Bvoco                                                          -0.137
Mbvoc               

We calculate the Sun's coordinates and calculate the irradiance on the plane of array (POA)

In [6]:
# calculate Sun's coordinates
solar_position = location.get_solarposition(times=tmy.index)

tilt = 24 
orientation = 180 # pvlib sets orientation origin at North -> South=0

# calculate irradiante at the plane of the array (poa)
poa_irradiance = pvlib.irradiance.get_total_irradiance(surface_tilt=tilt,
                                                        surface_azimuth=orientation,
                                                        dni=tmy['dni'],
                                                        ghi=tmy['ghi'],
                                                        dhi=tmy['dhi'],
                                                        solar_zenith=solar_position['apparent_zenith'],
                                                        solar_azimuth=solar_position['azimuth'])
    
#save annual irradiation on the plane of array (POA) in kWh
annual_irradiation = 0.001*poa_irradiance['poa_global'].sum()


Assuming a PR of 80%, the number of PV modules to produce 3,000 kWh can be calculated as

In [7]:
PR=0.8
annual_energy_yield_per_module = module_pmp*(annual_irradiation/1000)*PR
N_modules = math.ceil(3000/annual_energy_yield_per_module)
generator_capacity = N_modules*module_pmp
print("Number of modules = " + str(N_modules))
print("PV generator nominal capacity = " + str(round(generator_capacity))+' W')

Number of modules = 8
PV generator nominal capacity = 1929 W


We retrieve the inverter data from the SAM database.

In [8]:
sapm_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')
inverter = sapm_inverters['ABB__PVI_3_0_OUTD_S_US__208V_']
inverter

Vac                          208
Pso                    18.166279
Paco                      3000.0
Pdco                  3142.30127
Vdco                       310.0
C0                     -0.000008
C1                     -0.000011
C2                      0.000999
C3                     -0.000287
Pnt                          0.1
Vdcmax                     480.0
Idcmax                 10.136456
Mppt_low                   100.0
Mppt_high                  480.0
CEC_Date                     NaN
CEC_Type     Utility Interactive
Name: ABB__PVI_3_0_OUTD_S_US__208V_, dtype: object

For the temperature parameters, we assume an open rack glass-glass configuration. The maximum and minimum cell temperature can be calculated as

In [9]:
temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

# calculate the solar cell temperature
cell_temperature = pvlib.temperature.sapm_cell(poa_irradiance['poa_global'],
                                               tmy["temp_air"],
                                               tmy["wind_speed"],
                                               **temperature_model_parameters,)

T_c_max=max(cell_temperature)
T_c_min=min(cell_temperature)

print("Maximum cell temperature = " + str(round(T_c_max,1)) + " C")
print("Minimum cell temperature = " + str(round(T_c_min,1)) + " C")

Maximum cell temperature = 68.1 C
Minimum cell temperature = 1.2 C


To evaluate the number of PV modules connected in series, we need to calculate the maximum open-circuit voltage, as well as the maximum and minimum values for the voltage at the maximum power point.

In [10]:
Voc_max=module['Voco']*(1+module['Bvoco']/100*(T_c_min-20))

Vmp_max=module['Vmpo']*(1+module['Bvmpo']/100*(T_c_min-20))
Vmp_min=module['Vmpo']*(1+module['Bvmpo']/100*(T_c_max-20))

The maximum inverter input voltage determines the maximum number of PV modules that can be conected in series.

In [11]:
max_modules_in_series = inverter['Vdcmax']/Voc_max
math.floor(max_modules_in_series)

12

The MPPT input range determines the maximum and minimum number of PV modules that can be connected in series. 

In [12]:
max_modules_in_series = inverter['Mppt_high']/Vmp_max
math.floor(max_modules_in_series)

16

In [13]:
min_modules_in_series = inverter['Mppt_low']/Vmp_min
math.ceil(min_modules_in_series)

4

Since, we need to use 8 modules, we can connect all of them in a series-connected string. The current of the string will be determined by the short-circuit current of one module. We can check that this is below the inverter maximum input current. 

In [14]:
module['Isco']

8.8449

In [15]:
inverter['Idcmax']

10.136456